# Customized soil queries and input files
- info on NRCS SSURGO soil database: https://www.nrcs.usda.gov/wps/portal/nrcs/detail/soils/survey/geo/?cid=nrcs142p2_053631
- feed lat/lon info of each site into soilquery.py to retrieve soil info
- store output for each location

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import seaborn as sns
import plotly.express as px

from soilquery import soilquery

### Step 1: setup and soil data query
Query only needs to be executed once. <br/>
If executed, move on to step 3.

#### Read in site summary info

In [2]:
df_summary = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/site_summary.csv', 
                         index_col=0, dtype={'site':str})
df_siteyears = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/siteyears_filtered.csv', 
                            index_col=0, dtype={'site':str})
sites = set(df_siteyears.site)
df_sites = df_summary[df_summary.site.isin(sites)]
df_sites.head()

,site,class,station,state,tzone,lat,lon,years,area,perct_irri
6,722070,1,SAVANNAH INTL AP,GA,-5,32.117,-81.200,22,2445.923794,6.585904
7,722080,1,CHARLESTON INTL ARPT,SC,-5,32.900,-80.033,25,10052.406880,5.735219
9,722140,1,TALLAHASSEE REGIONAL AP [ISIS],FL,-5,30.400,-84.350,19,9510.127270,13.348148
10,722170,1,MACON MIDDLE GA REGIONAL AP,GA,-5,32.683,-83.650,22,2731.632292,18.039572
11,722180,1,AUGUSTA BUSH FIELD,GA,-5,33.367,-81.967,18,5940.794366,8.403471


#### Query soil info for all weather station locations

In [3]:
lats = df_sites.lat
lons = df_sites.lon
df_soils = pd.DataFrame()

for site, lat, lon in zip(sites, lats, lons):
    try:
        df_soil = soilquery(round(lat,2), round(lon,2))
        df_soil['site'] = site
        df_soil['lat'] = lat
        df_soil['lon'] = lon
        df_soils = df_soils.append(df_soil)
    except:
        print(site, 
              df_sites[df_sites.site==site].station.values[0], 
              df_sites[df_sites.site==site].state.values[0])

df_soils.dropna(axis=0, inplace=True)
df_soils.reset_index(drop=True, inplace=True)
df_soils.head()

725370 DETROIT METROPOLITAN ARPT MI
725290 ROCHESTER GREATER ROCHESTER I NY
726360 MUSKEGON COUNTY ARPT MI
722080 CHARLESTON INTL ARPT SC
724290 DAYTON INTERNATIONAL AIRPORT OH
725330 FORT WAYNE INTL AP IN
725040 BRIDGEPORT SIKORSKY MEMORIAL CT
725246 MANSFIELD LAHM MUNICIPAL ARPT OH
724089 WILMINGTON NEW CASTLE CNTY AP DE
722670 LUBBOCK INTERNATIONAL AP TX
725620 NORTH PLATTE REGIONAL AP NE
724236 JACKSON JULIAN CARROLL AP KY
724390 SPRINGFIELD CAPITAL AP IL
724510 DODGE CITY REGIONAL AP KS
725470 DUBUQUE REGIONAL AP IA
723140 CHARLOTTE DOUGLAS INTL ARPT NC
722530 SAN ANTONIO INTL AP TX


,cokey,chkey,prcent,slope_r,slope,hzname,depth,awc,clay,silt,sand,OM,dbthirdbar,th33,bd,site,lat,lon
0,20681154,60804747,60.0,1,2,H1,0.0,0.07,7.0,6.6,86.4,1.50,1.55,16.6,-0.1505,722590,32.117,-81.20
1,20681154,60804748,60.0,1,2,H2,71.0,0.11,25.0,18.0,57.0,0.25,1.63,24.9,-0.2327,722590,32.117,-81.20
2,20681154,60804749,60.0,1,2,H3,150.0,0.11,27.5,17.4,55.1,0.25,1.63,25.9,-0.2427,722590,32.117,-81.20
3,20646330,60671139,85.0,3,5,A,0.0,0.04,1.5,4.9,93.6,0.53,1.45,5.2,-0.0375,725118,30.400,-84.35
4,20646330,60671138,85.0,3,5,C,18.0,0.04,1.0,1.1,97.9,0.05,1.45,4.0,-0.0255,725118,30.400,-84.35


#### Query soil info for all NASS locations
- A lot more sites to query for all NASS locations.
- The reason for querying all this data is to hopefully have a more realistic and smoothed representation of soil texture near the weather station locations, since most of the weather stations are not farming sites. NASS data are directly from farm sites.
- Would require additional processing to select maybe 5 nearest NASS sites to the weather stations and produce averaged soil texture data.

In [ ]:
df_obs = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/obs_areairri.csv', index_col=0)
lats = df_obs.lat
lons = df_obs.lon
df_soils = pd.DataFrame()

for lat, lon in zip(lats, lons):
    try:
        df_soil = soilquery(round(lat,2), round(lon,2))
        df_soil['lat'] = lat
        df_soil['lon'] = lon
        df_soils = df_soils.append(df_soil)
    except:
        print(lat, lon)

df_soils.dropna(axis=0, inplace=True)
df_soils = df_soils[df_soils.sand + df_soils.silt + df_soils.clay == 100] # include only sites where
                                                                          # texture sums to 100
df_soils.reset_index(drop=True, inplace=True)
df_soils.head()

### Step 2: Add additional info to raw soil data

In [158]:
df_soils = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/soiltexture_nass.csv', 
                       index_col=0, dtype={'depth_category':str})
df_textures = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/textures.csv',index_col=0)
df_textures.head()

,Cl,SiCl,SaCl,ClLo,SiClLo,SaClLo,Lo,SiLo,SaLo,Si,LoSa,Sa
1,0,0,0,0,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0


#### Bin soil quert outputs into 5 soil depth categories

In [159]:
df_soils['depth_category'] = np.nan
df_soils.loc[(df_soils.depth > -1) & (df_soils.depth <= 0),'depth_category'] = 0
df_soils.loc[(df_soils.depth > 0) & (df_soils.depth <= 50), 'depth_category'] = 50
df_soils.loc[(df_soils.depth > 50) & (df_soils.depth <= 100), 'depth_category'] = 100
df_soils.loc[(df_soils.depth > 100) & (df_soils.depth <= 150), 'depth_category'] = 150
df_soils.loc[df_soils.depth >150, 'depth_category'] = 200

#### Assign soil texture
Soil textures assigned using R package 'soiltexture'. <br/>
Done locally through RStudio, output textures.csv copied to server.

In [160]:
textures = []
for i in np.arange(df_textures.shape[0]):
    texture = df_textures.iloc[i]
    try:
        texture = texture[texture == 1].index[0]
        textures.append(texture)
    except:
        texture = 'ambiguous' # texture right on boarder of two groups
        textures.append(texture)
        
df_soils['texture'] = textures

#### Group soil texture into broader soil groups
See Table 4: <br/> http://www.fao.org/tempref/FI/CDrom/FAO_Training/FAO_Training/General/x6706e/x6706e06.htm

In [161]:
# create column for soil group
df_soils['sgroup'] = ''
df_soils['sgroup_note'] = ''

# soil group 1: sandy soils (coarse texture)
soil_category = ((df_soils.texture == 'Sa') | (df_soils.texture == 'LoSa'))
df_soils.loc[soil_category, 'sgroup'] = 1
df_soils.loc[soil_category, 'sgroup_note'] = 'coarse'

# soil group 2: loamy soils (moderately coarse texture)
soil_category = ((df_soils.texture == 'SaLo'))
df_soils.loc[soil_category, 'sgroup'] = 2
df_soils.loc[soil_category, 'sgroup_note'] = 'moderately coarse'

# soil group 3: loamy soils (medium texture)
soil_category = ((df_soils.texture == 'Lo') | (df_soils.texture == 'SiLo') | 
                 (df_soils.texture == 'Si'))
df_soils.loc[soil_category, 'sgroup'] = 3
df_soils.loc[soil_category, 'sgroup_note'] = 'medium'

# soil group 4: loamy soils (moderately fine texture)
soil_category = ((df_soils.texture == 'ClLo') | (df_soils.texture == 'SaClLo') |
                 (df_soils.texture == 'SiClLo'))
df_soils.loc[soil_category, 'sgroup'] = 4
df_soils.loc[soil_category, 'sgroup_note'] = 'moderately fine'

# soil group 5: clayey soils (fine texture)
soil_category = ((df_soils.texture == 'SaCl') | (df_soils.texture == 'SiCl') |
                 (df_soils.texture == 'Cl'))
df_soils.loc[soil_category, 'sgroup'] = 5
df_soils.loc[soil_category, 'sgroup_note'] = 'fine'

### Step 4: Output soil info

In [163]:
#df_soils.to_csv('/home/disk/eos8/ach315/upscale/weadata/soiltexture.csv')
#df_soils.to_csv('/home/disk/eos8/ach315/upscale/weadata/soiltexture_nass.csv')